In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import elmo
%load_ext autoreload

In [2]:
%autoreload 1
import importlib
from utils import tokenize
from utils import Utils
from utils import find_word
from utils import find_word_root
import utils
from fetched_word import FetchedWord
import config
from webpage import Webpage
import inference_tools

In [3]:
driver = config.DRIVER
elmo = elmo.Elmo()
tools = Utils()

Using real Elmo?


### Opens the file that contains rare skill words. 

- Returns a list of custom objects called SkillWords. 
- Each object is an uncommon word(rare word) that is extracted from the knowledge base fetched from wikipedia.
- This object along with word contains the following : 
* "Word root", 
* "Sentence in which it was used",
* "Embeddings from elmo",  
* "Indices of their presence in sentences" ,
* "which skill it belongs to", 
* "number of times it occured(count)"

In [4]:
with open(config.SKILL_WORD_PATH, 'rb') as handle:
    skill_rare_words = pickle.load(handle)

#### Opens the file that contains synthetically prepared dataset with urls that are related to topics such as 

- Shopping
- Entertainment 
- Travel 

In [5]:
other_df = pd.read_excel("../data/web_history/other_website_history_xl.xls")

In [6]:
other_df.head()

,URL,Title,Visit Time,Visit Count,Visited From,Visit Type,Web Browser,User Profile,Browser Profile,URL Length,Typed Count,History File,Record ID
0,https://ohthatfilmblog.com/,Oh! That Film Blog | The ramblings of a self c...,2021-12-03 14:04:00,1,NaN,Link,Chrome,welcome,Default,27,0,C:\Users\welcome\AppData\Local\Google\Chrome\U...,12382
1,https://www.themovieblog.com/,The Movie Blog | The Official Home of Correct ...,2021-12-03 14:04:00,1,NaN,Link,Chrome,welcome,Default,29,0,C:\Users\welcome\AppData\Local\Google\Chrome\U...,12381
2,https://www.movie-blogger.com/,Movie-Blogger.com | The No.1 Independent Movie...,2021-12-03 14:04:00,1,NaN,Link,Chrome,welcome,Default,30,0,C:\Users\welcome\AppData\Local\Google\Chrome\U...,12380
3,https://blogging.org/top-movie-blogs/,Top 10 Movie Blogs on the Internet Today | Mov...,2021-12-03 14:04:00,1,NaN,Link,Chrome,welcome,Default,37,0,C:\Users\welcome\AppData\Local\Google\Chrome\U...,12379
4,https://thefilm.blog/,The Film Blog | The official blog of everythin...,2021-12-03 14:04:00,1,NaN,Link,Chrome,welcome,Default,21,0,C:\Users\welcome\AppData\Local\Google\Chrome\U...,12378


#### Opens the file that contains synthetically prepared dataset with urls that are related to an example skill set

In [7]:
skills_df = pd.read_excel("../data/web_history/synthetic_data_490_xl.xls")

In [8]:
skills_df.head()

,URL,TITILE,Visited_time,1,2,3,4,5,6,7,8,9,10
0,https://www.tensorflow.org/tutorials/images/cnn,Convolutional Neural Network (CNN) | TensorF...,2021-12-03 03:39:00,2.0,NaN,Link,Chrome,welcome,Default,47.0,0.0,C:\Users\welcome\AppData\Local\Google\Chrome\U...,12121
1,https://www.tensorflow.org/,TensorFlow,2021-12-03 03:39:00,2.0,NaN,Link,Chrome,welcome,Default,27.0,0.0,C:\Users\welcome\AppData\Local\Google\Chrome\U...,12120
2,https://en.wikipedia.org/wiki/TensorFlow,TensorFlow - Wikipedia,2021-12-03 03:38:00,1.0,NaN,Link,Chrome,welcome,Default,40.0,0.0,C:\Users\welcome\AppData\Local\Google\Chrome\U...,12119
3,https://github.com/tensorflow/tensorflow,tensorflow/tensorflow: An Open Source Machine ...,2021-12-03 03:38:00,1.0,NaN,Link,Chrome,welcome,Default,40.0,0.0,C:\Users\welcome\AppData\Local\Google\Chrome\U...,12118
4,https://www.tensorflow.org/,TensorFlow,2021-12-03 03:38:00,2.0,NaN,Link,Chrome,welcome,Default,27.0,0.0,C:\Users\welcome\AppData\Local\Google\Chrome\U...,12117


In [9]:
urls = list(other_df['URL'])
for i, url in enumerate(urls):
    if i == 1:
        break
    _, fetched_words, _ = inference_tools.test_url(elmo, url)
    
    

Processing Text from the web page
